In [1]:
## 이 장에서는 앞에서 구현한 RNN과 LSTM을 이용해 애플리케이션을 구현해 본다.
## 바로 언어 모델을 사용해 '문장 생성'을 수행할 것이다.
## 말뭉치를 통해 학습한 언어 모델을 이용해 새로운 문장을 만들어 낸다.

# 1. 언어 모델을 사용한 문장 생성

In [4]:
## 지금까지 우리는 언어 모델을 다뤄왔다.
## 언어 모델은 다양한 애플리케이션에서 활용할 수 있다. 
## 대표적인 예로 기계 번역, 음성 인식, 문장 생성 등이 있다.

### RNN을 사용한 문장 생성의 순서

In [5]:
## 말뭉치로 학습을 수행한 언어 모델에 단어를 입력하면 이 언어 모델은 다음에 출현할 단어의 확률분포를 출력한다.
## 예를 들어 you say goodbye and i say hello.라는 말뭉치로 학습한 언어 모델에 i라는 단어를 넣으면 
## you, say, goodbye, and, i, hello, . 각 단어에 대한 확률이 출력된다.

## 이 결과를 기초로 다음 단어를 생성하려면 어떻게 해야할까?
## 첫 번째로, 확률이 가장 높은 단어를 선택하는 방법이 있다.(결정적)
## 두 번째로, 각 후보 단어의 확률에 맞게 선택하는 방법이 있다.(확률적)
## 확률이 높은 단어는 선택되기 쉽고, 낮은 단어는 선택되기 어렵다.
## 이 방식에서는 선택 되는 단어가 매번 다를 수 있다.

## 여기서는 확률적 방법으로 문장을 생성해볼 것이다.

In [6]:
## 앞의 예에서 say라는 단어가 (확률적으로) 선택되었다고 해보자
## 계속해서 두 번째 단어를 샘플링 해보자. 
## 방금 생성한 단어인 say를 언어 모델에 입력하여 다음 단어의 확률분포를 얻는다.
## 그리고 그 확률분포를 기초로 다시 단어를 샘플링 한다.
## 이 작업을 원하는 만큼 반복한다. 그러면 새로운 문장을 생성할 수 있다.

In [7]:
## 주목할 점은 여기서 생성된 문장은 훈련 데이터에는 존재하지 않는 새로운 문장이다.
## 언어 모델이 말뭉치로부터 단어의 출현 패턴을 올바르게 학습할 수 있다면 
## 여기서 생성된 문장은 인간에게도 자연스럽고 의미가 통하는 문장일 것이다.

### 문장 생성 구현

In [8]:
## 앞 장에서 구현한 언어 모델인 Rnnlm 클래스를 상속해 RnnlmGen 클래스를 만들고, 이 클래스에 문장 생성 메서드를 추가한다.

In [9]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm


class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)


class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state)

In [28]:
## 문장 생성

# coding: utf-8
import sys, os
sys.path.append('..')
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
#model.load_params('Rnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)


you epicenter crowded way fined belgian adequate fitness setback rebounded racing medicine images most heating issuer backed beer respectability tro kume competent careful regulate contrasts fluor diego rocky carpet thieves incur metropolitan cautioned sustain indicate basically chung magic scoring lowe pressed exchange game chair idle created bronx introduction acceptance injuries restrain generations pieces noranda free-market brazil spokesman characters brink prestige shearson unused told constituents loosen ample irresponsible ancient my allowance imposes disclosing hasbro kuala craig guards final plc ministry booklets leading smokers mix portion coca-cola military reinforcement cheered contact creatures disappointed development publicized franklin rubles ga. typical successor youngest initiative


In [29]:
## 결과를 보면 단어들을 엉터리로 나열한 글이 출력되었다.
## 모델의 가중치 초깃값으로 무작위한 값을 사용했기 때문이다.

## 그러면 학습을 수행한 언어 모델이라면 어떻게 다를까?
## 학습을 끝낸 가중치를 이용해 문장을 생성해보자
## 주석 처리해둔 model.load_params('Rnnlm.pkl')을 주석 해제하면 학습한 가중치 매개변수를 읽어들인다.

In [30]:
## 문장 생성

# coding: utf-8
import sys, os
sys.path.append('..')
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('Rnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)


you appear pleased with the activities.
 the hair marine is due to serve as on a direct city headquarters.
 in mr. honecker who they will pay about about spending.
 although he recommend why many companies says oliver nyse on the northern uncle.
 over the faded mr. antar signed his teacher they get his potentially chemistry stated for a decision to invest up to the offer completion to move.
 so he said.
 the claims of the pacific mixte share gained short to nearly half quickly.
 a analyst for the biggest security restaurant spokesmen for


In [31]:
## 문법적으로 이상하거나 의미가 통하지 않는 문장이 섞여 있지만, 그럴듯한 문장도 있다.
## 이처럼 학습된 매개변수로 생성한 문장은 어느 정도 올바른 문장이라고 할 수 있다.
## 하지만 부자연스러운 문장도 많으니 개선의 여지가 있다.
## 더 자연스러운 문장을 생성하기 위해서는 더 나은 언어 모델을 사용해야 한다.

### 더 좋은 문장으로

In [32]:
## 앞에서 사용한 언어 모델은 Rnnlm이라는 클래스로 구현한 언어 모델이었다.
## 우리는 앞 장에서 언어 모델을 개선하여 퍼플렉서티를 135에서 75까지 개선했다.
## 개선된 언어 모델인 BetterRnnlm 클래스를 이용해 문장을 생성해보자

In [41]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *
from ch07.rnnlm_gen import BetterRnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)


model = BetterRnnlmGen()
model.load_params('BetterRnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')

print(txt)


model.reset_state()

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x)

word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)


you would stop.
 daffynition.
 mr. moss was one of the first extensive legal leaders on his findings among the money strategist and the roper organization in the state.
 in september we expect the position of hard evidence of recruit a commission max 's failure to leave the courts at a time.
 this situation because much are the process of unscrupulous culture and civil rights to any everybody 's political prescription interests.
 we know that the environmentalists do that even before they make it back to the same kind of hearings they give the academy instead
--------------------------------------------------
the meaning of life is reluctant to occur.
 labor and big programs affects the horn and areas will pass.
 about half of the public market 's fund intends to tough to modernize the game and an innovative buyer of the baltimore carrier and the west german valley union the u.s. outlays the u.s. lender could go to the commerce department.
 the agreement wo n't directly be substantiall

In [42]:
## 좀 더 자연스러운 문장이 생성된 것 같다.

## 마지막으로 우리의 모델에 'the meaning of life is'라는 글을 주고 이어지는 말을 생성해보자.

In [46]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *
from ch07.rnnlm_gen import BetterRnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)


model = BetterRnnlmGen()
model.load_params('BetterRnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')

print(txt)


model.reset_state()

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x)

word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)


you are very important.
 so is the rule of vested sectors of the broader depression.
 the negative deterioration of u.s. markets is a success.
 the japanese government has anticipate a bitter slowing in u.s. profits in the u.s. and europe with heavy investment pressures but perhaps in japan the trade figures has deteriorated largely because of the continued weakness in foreign currency sales.
 western european real estate developer and u.s. officials are designed to absorb the usual natural gas import.
 mr. dorrance says he wants little to the more point in the region 's best
--------------------------------------------------
the meaning of life is the real york marriage of its seventh avenue of the best sweep of the 1960s game.
 the battery of the urban football commitment did n't affect even in the district of original trucks reflecting passenger.
 mr. pilson said that more issues the motel division is talking about being involved in toronto-based immune deficiency syndrome noting th